In [2]:
# built-in libraries
import sys
import math
import glob
import traceback

# keras
import keras
from keras import layers

# numpy
import numpy as np

# tensorflow
import tensorflow as tf

# matplotlib
from matplotlib import pyplot as plt

# sente
import sente
from sente import sgf

# tqdm
import tqdm

Initialize the network

Network Architecture originally based on The original AlphaGo Lee Architecture

> "The input to the policy network is a 19 × 19 × 48 image stack consisting of 48 feature planes. The first
> hidden layer zero pads the input into a 23 × 23 image, then convolves k filters of kernel size 5 × 5 with
> stride 1 with the input image and applies a rectifier nonlinearity. Each of the subsequent hidden layers 2 to
> 12 zero pads the respective previous hidden layer into a 21 × 21 image, then convolves k filters of kernel size
> 3 × 3 with stride 1, again followed by a rectifier nonlinearity. The final layer convolves 1 filter of kernel
> size 1 × 1 with stride 1, with a different bias for each position, and applies a softmax func- tion. The match
> version of AlphaGo used k = 192 filters; Fig. 2b and Extended Data Table 3 additionally show the results of
> training with k = 128, 256 and 384 filters."

A few modifications have been made

- Only has 4 input Features instead of 48
- Uses batch normalization instead of dropout

Initialize the Data Generator to train the network with.

We have 3 basic steps here:
1) Create a generator for 19x19 games
    1) Glob the files so we have a complete list of them
    2) Filter out any games that are not 19x19
    3) Filter out any games that have invalid moves (according to sente)
2) Extract the features and label
    1) Iterate through the files

In [3]:
# get the numpy spec from a generic game
input_numpy = sente.Game().numpy()

# input layer
input_layer = layers.Input(shape=input_numpy.shape)
x = input_layer

# First layer has a kernel size of 5

x = layers.Conv2D(filters=192,
                  kernel_size=5,
                  padding="same",
                  activation="relu"
                  )(input_layer)
x = layers.BatchNormalization()(x)

# subsequent layers have kernel sizes of 3
for i in range(11):
    x = layers.Conv2D(filters=192,
                      kernel_size=3,
                      activation="relu",
                      padding="same")(x)
    x = layers.BatchNormalization()(x)

# output layer adds everything together with bias
x = layers.Conv2D(filters=1,
                  kernel_size=1,
                  activation="relu")(x)
x = layers.Flatten()(x)
output = layers.Softmax()(x)

model = keras.Model(inputs=input_layer, outputs=output, name="Policy-Network")
# keras.utils.plot_model(model)

model.summary()

Metal device set to: Apple M1


2022-11-12 11:08:40.514453: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-12 11:08:40.514999: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "Policy-Network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 19, 4)]       0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 192)       19392     
                                                                 
 batch_normalization (BatchN  (None, 19, 19, 192)      768       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 192)       331968    
                                                                 
 batch_normalization_1 (Batc  (None, 19, 19, 192)      768       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 192)    

In [11]:
def sgf_generator(glob_string):
    """

    create an SGF file generator.

    :param glob_string: string to glob files with
    :return: a generator yielding
    """

    # obtain a generator for the glob
    files = glob.iglob(glob_string)

    for file in files:

        try:
            game = sgf.load(file)
            # game.play_default_sequence()

            # if the board is not 19x19, skip
            if game.numpy().shape != (19, 19, 4):
                continue
            else:
                # yield the game
                yield game
        except (sente.exceptions.InvalidSGFException,
                sente.exceptions.IllegalMoveException,
                ValueError) as error:
            traceback.print_exception(error, file=sys.stderr)
            continue


generator = sgf_generator("sgfs-uploaded/*/*/*/*")

In [5]:
def get_8_fold_symmetries_x(game: sente.Game):
    """

    generates a numpy array containing a sente game, expanded to all of its 8-fold symmetries

    :param game: the game to get the symmetries for
    :return: 8x19x19x4 array containing all duplicates
    """

    board = game.numpy()
    transpose = np.transpose(board, axes=(1, 0, 2))
    x = np.empty((8,) + board.shape)

    x[0, :, :, :] = board
    x[1, :, :, :] = np.flip(board, axis=0)
    x[2, :, :, :] = np.flip(board, axis=1)
    x[3, :, :, :] = np.flip(board, axis=(0, 1))
    x[4, :, :, :] = transpose
    x[5, :, :, :] = np.flip(transpose, axis=0)
    x[6, :, :, :] = np.flip(transpose, axis=1)
    x[7, :, :, :] = np.flip(transpose, axis=(0, 1))

    return x


def get_8_fold_symmetries_y(move: np.array):
    """

    generates a numpy array containing a sente game, expanded to all of its 8-fold symmetries

    :param game: the game to get the symmetries for
    :return: 8x19x19x4 array containing all duplicates
    """

    transpose = np.transpose(move)
    x = np.empty((8,) + move.shape, dtype=np.ubyte)

    x[0, :, :] = move
    x[1, :, :] = np.flip(move, axis=0)
    x[2, :, :] = np.flip(move, axis=1)
    x[3, :, :] = np.flip(move, axis=(0, 1))
    x[4, :, :] = transpose
    x[5, :, :] = np.flip(transpose, axis=0)
    x[6, :, :] = np.flip(transpose, axis=1)
    x[7, :, :] = np.flip(transpose, axis=(0, 1))

    x = np.reshape(x, (8, 19 * 19))

    return x


In [6]:
def training_data_generator(glob_string: str):
    """

    creates a training data generator object

    :return:
    """

    file_generator = sgf_generator(glob_string)

    batch_size = 64

    active_games = set()
    active_games_size = batch_size // 8

    # go through all the moves in the game
    while True:

        # remove any games in which we either encounter an illegal move or there are no more moves
        for game in active_games.copy():
            next_branches = game.get_branches()

            # remove the game if it has no more moves or the next move is illegal
            if len(next_branches) == 0:
                active_games.remove(game)
                continue
            if not game.is_legal(next_branches[0]):
                active_games.remove(game)

        # fill active games if we can
        while len(active_games) < active_games_size:
            next_game = next(file_generator, None)

            # only add to active games
            if next_game is None:
                # reset the generator
                print("no more games to train on", file=sys.stderr)
                return

            branches = next_game.get_branches()

            if len(branches) != 0:
                # only add the move if the next branch is legal
                if next_game.is_legal(next_game.get_branches()[0]):
                    active_games.add(next_game)

        # go through all the active games and add the moves to the result

        # first, compute the size of the batch we are currently operating on
        current_batch_size = len(active_games) * 8

        # initialize the empty arrays
        x = np.zeros(shape=(current_batch_size, 19, 19, 4))
        y = np.zeros(shape=(current_batch_size, 19 * 19))

        # go through all the active games and fill the arrays
        for i, game in enumerate(active_games):

            # obtain the branches
            branches = game.get_branches()
            move = branches[0]

            move_array = np.zeros(shape=(19, 19))
            move_array[move.get_x(), move.get_y()] = 1


            # set x by getting the 8-fold symmetries
            x[8 * i:8 * (i + 1)] = get_8_fold_symmetries_x(game)
            y[8 * i:8 * (i + 1)] = get_8_fold_symmetries_y(move_array)

            # play the move on the board
            game.play(move)

        # yield the results
        yield x, y

Fit the Model

In [9]:
# instantiate the generator
generator = training_data_generator("sgfs-uploaded/*/*/*/*")
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy")

history = model.fit(generator, epochs=25, steps_per_epoch=1000)

Epoch 1/25


/var/folders/8p/v3fpbly136b72tk4pw7qhr4r0000gp/T/ipykernel_441/2674019017.py:16: Warning: Unknown SGF Property: "JD"
  game = sgf.load(file)
2022-11-12 11:48:06.179723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 373/1000 [==========>...................] - ETA: 5:44 - loss: 3.8669

/var/folders/8p/v3fpbly136b72tk4pw7qhr4r0000gp/T/ipykernel_441/2674019017.py:16: Warning: The Property "AP" is not supported on this version of SGF (FF[1])
The file was automatically converted to FF[4]
  game = sgf.load(file)


1000/1000 [==============================] - 420s 417ms/step - loss: 3.1233
Epoch 2/25
 385/1000 [==========>...................] - ETA: 3:27 - loss: 2.0985

...

Traceback (most recent call last):
  File "/var/folders/8p/v3fpbly136b72tk4pw7qhr4r0000gp/T/ipykernel_441/2674019017.py", line 16, in sgf_generator
    game = sgf.load(file)
sente.exceptions.InvalidSGFException: Error loading file "sgfs-uploaded/2013/01/26/437625-Mahiraku-mdw602.sgf": The Property "LT" is not supported on this version of SGF (FF[4])


1000/1000 [==============================] - 449s 448ms/step - loss: 3.0142
Epoch 10/25
1000/1000 [==============================] - 356s 356ms/step - loss: 3.1139
Epoch 11/25
 466/1000 [============>.................] - ETA: 3:04 - loss: 3.0103

2022-11-12 13:09:37.528676: W tensorflow/core/framework/op_kernel.cc:1733] UNKNOWN: IllegalMoveException: 
Traceback (most recent call last):

  File "/Users/arthur/miniforge3/envs/Sedol-Go/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/arthur/miniforge3/envs/Sedol-Go/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/arthur/miniforge3/envs/Sedol-Go/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1030, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/arthur/miniforge3/envs/Sedol-Go/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 831, in wrapped_generator
    for data in generator_fn():

  File "/var/folders/8p/v3fpbly136b72tk4pw7qhr4r0000gp/T/ipykernel_441/1768145525.py", line 32, in training_data_generator
    next_game = n

 467/1000 [=============>................] - ETA: 3:04 - loss: 3.0111

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  IllegalMoveException: 
Traceback (most recent call last):

  File "/Users/arthur/miniforge3/envs/Sedol-Go/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/arthur/miniforge3/envs/Sedol-Go/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/arthur/miniforge3/envs/Sedol-Go/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1030, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/arthur/miniforge3/envs/Sedol-Go/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 831, in wrapped_generator
    for data in generator_fn():

  File "/var/folders/8p/v3fpbly136b72tk4pw7qhr4r0000gp/T/ipykernel_441/1768145525.py", line 32, in training_data_generator
    next_game = next(fil

Plot a Summary and Save weights



In [ ]:
model.save("policy network")